<a href="https://colab.research.google.com/github/RajeshMallik19/Project_Q/blob/student_dropout/Student_3_Deep_Learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import SMOTE
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Bidirectional, LSTM, Conv1D, MaxPooling1D, Flatten, Dense, Dropout

# Load the prepped data
df = pd.read_csv('/content/student_dropout_prepped.csv')

# Separate features (X) and target (y)
X = df.drop(columns=['at_risk'])
y = df['at_risk']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

# Apply SMOTE to balance the training data
smote = SMOTE(random_state=42)
X_train_resampled, y_train_resampled = smote.fit_resample(X_train, y_train)

print("Data loaded and prepped for Deep Learning.")

Data loaded and prepped for Deep Learning.


In [3]:
# Convert dataframes to numpy arrays with float dtype before reshaping
X_train_resampled_np = X_train_resampled.values.astype(float)
X_test_np = X_test.values.astype(float)

# Reshape the data to be 3D
# We treat our tabular data as a sequence with a single timestep.
X_train_reshaped = np.reshape(X_train_resampled_np, (X_train_resampled_np.shape[0], 1, X_train_resampled_np.shape[1]))
X_test_reshaped = np.reshape(X_test_np, (X_test_np.shape[0], 1, X_test_np.shape[1]))


# --- Verification Step ---
print(f"Original training data shape: {X_train_resampled_np.shape}")
print(f"Reshaped training data shape: {X_train_reshaped.shape}")
print(f"Original testing data shape: {X_test_np.shape}")
print(f"Reshaped testing data shape: {X_test_reshaped.shape}")

Original training data shape: (564, 60)
Reshaped training data shape: (564, 1, 60)
Original testing data shape: (96, 60)
Reshaped testing data shape: (96, 1, 60)


In [4]:
# --- Define the Model Architecture ---
model = Sequential([
    # Input shape is (timesteps, features)
    Bidirectional(LSTM(64, return_sequences=True), input_shape=(X_train_reshaped.shape[1], X_train_reshaped.shape[2])),
    Conv1D(filters=32, kernel_size=1, activation='relu'),
    MaxPooling1D(pool_size=1),
    Flatten(),
    Dense(32, activation='relu'),
    Dropout(0.5), # Helps prevent overfitting
    Dense(1, activation='sigmoid') # Output layer for binary classification
])

# --- Compile the Model ---
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

# Print a summary of the model
model.summary()

# --- Train the Model ---
print("\nTraining the BiLSTM-CNN model...")
history = model.fit(
    X_train_reshaped,
    y_train_resampled,
    epochs=20,          # We'll train for 20 passes over the data
    batch_size=32,
    validation_split=0.1, # Use 10% of training data for validation
    verbose=1           # Show the progress
)

/usr/local/lib/python3.12/dist-packages/keras/src/layers/rnn/bidirectional.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ bidirectional (Bidirectional)   │ (None, 1, 128)         │        64,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d (Conv1D)                 │ (None, 1, 32)          │         4,128 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d (MaxPooling1D)    │ (None, 1, 32)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 32)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 32)             │         1,056 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 32)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 69,217 (270.38 KB)

 Trainable params: 69,217 (270.38 KB)

 Non-trainable params: 0 (0.00 B)


Training the BiLSTM-CNN model...
Epoch 1/20
16/16 ━━━━━━━━━━━━━━━━━━━━ 7s 48ms/step - accuracy: 0.5953 - loss: 0.6770 - val_accuracy: 0.7018 - val_loss: 0.5779
Epoch 2/20
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.8014 - loss: 0.5570 - val_accuracy: 0.8421 - val_loss: 0.4157
Epoch 3/20
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.8299 - loss: 0.4789 - val_accuracy: 0.8421 - val_loss: 0.3599
Epoch 4/20
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - accuracy: 0.8598 - loss: 0.4057 - val_accuracy: 0.8947 - val_loss: 0.2912
Epoch 5/20
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.8391 - loss: 0.3822 - val_accuracy: 0.9298 - val_loss: 0.2585
Epoch 6/20
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.8880 - loss: 0.3454 - val_accuracy: 0.9649 - val_loss: 0.2199
Epoch 7/20
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.8973 - loss: 0.3100 - val_accuracy: 0.8596 - val_loss: 0.3080
Epoch 8/20
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - accuracy: 0.9144 - loss:

In [5]:
from sklearn.metrics import classification_report

# Make predictions on the test set
# The model outputs probabilities, so we round to get 0 or 1
y_pred_dl_proba = model.predict(X_test_reshaped)
y_pred_dl = (y_pred_dl_proba > 0.5).astype(int)

# --- Evaluation Step ---
print("\n--- BiLSTM-CNN Model Evaluation ---")
print(classification_report(y_test, y_pred_dl))

3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step 

--- BiLSTM-CNN Model Evaluation ---
              precision    recall  f1-score   support

           0       0.92      0.99      0.95        71
           1       0.95      0.76      0.84        25

    accuracy                           0.93        96
   macro avg       0.94      0.87      0.90        96
weighted avg       0.93      0.93      0.92        96

